In [17]:
import numpy as np
import pandas as pd
import datetime
from talib import abstract#support pandas.series
import tushare as tu

import plotly.plotly as py
import cufflinks as cf
# Cufflinks binds plotly to pandas dataframes in IPython notebook. Read more
import plotly.graph_objs as go
from plotly import tools
cf.set_config_file(world_readable=True,offline=False)

todaydate=datetime.datetime.now()
start=todaydate-datetime.timedelta(days=200)
stock_name='601668'
base_stock = tu.get_hist_data(stock_name,start=start.strftime('%Y-%m-%d'))

In [66]:
class stockSeries(pd.core.frame.DataFrame):
    def sma(self, window):
        if len(self) < window:
            return None
        result=map(lambda x:sum(self['close'].iloc[-x:(-x+window)]) / float(window)
                   , [i for i in range(len(self)+1) if i>window])
        return pd.Series(result[::-1],index=self.index[:-window])
    def ema_genereator(self, window):
        c = 2.0 / float(window + 1)
        ema_t=sum(self['close'].iloc[-window:]) / float(window)
        t=window+1
        while True:
            yield ema_t
            ema_t= c*self['close'].iloc[-t]+(1-c)*ema_t
            t=t+1
    def ema(self,window):
        g=self.ema_genereator(window)
        result= [g.next() for i in range(len(self)-window+1)]
        return pd.Series(result[::-1],index=self.index[:-window+1])
    def macd(self,fasterwindow=12,slowerwindow=26,signalwindow=9):
        result= pd.DataFrame({fasterwindow:self.ema(fasterwindow),
                              slowerwindow:self.ema(slowerwindow)}).sort_index(axis=0 ,ascending=False)
        result['macd']=result[fasterwindow]-result[slowerwindow]
        temp=stockSeries({'close':result['macd']})
        temp=temp.dropna()
        temp=stockSeries({'close':temp['close']})
        result[signalwindow]=temp.ema(signalwindow)
        
        result['Signal'] = result['macd']-result[signalwindow]
        return result
    def 
    
    def plot_macd(self,fasterwindow=12,slowerwindow=26,signalwindow=9,ohlcinclude=False):
        result=self.macd(fasterwindow,slowerwindow,signalwindow)
        if ohlcinclude:
            fig = tools.make_subplots(rows=2, cols=1,shared_xaxes=True, shared_yaxes=True)
            trace1 = go.Candlestick(x=self.index,
                                   open=self.open,
                                   high=self.high,
                                   low=self.low,
                                   close=self.close)
            trace2= go.Scatter(x=result.index,
                              y=result.macd,name='macd')
            trace3= go.Scatter(x=result.index,
                              y=result[signalwindow],name='9EMA')
            trace4=go.Bar(x=result.index,
                         y=result.macd-result[signalwindow],name='macd-9ema')
            fig.append_trace(trace1, 1, 1)
            fig.append_trace(trace2, 2, 1)
            fig.append_trace(trace3, 2, 1)
            fig.append_trace(trace4, 2, 1)            
            fig['layout'].update(height=800, width=1000, title=stock_name)
            return py.iplot(fig)
        else:
            return result[['macd',signalwindow]].iplot(kind='scatter')
        
        
        

In [68]:
a=stockSeries({'close':base_stock['close'],'open':base_stock['open'],
               'high':base_stock['high'],'low':base_stock['low']})
a.head()
len(a)
a.sma(window=5)
a.ema(window=26)
a.plot_macd(12,26,9,True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

